In [1]:
import requests
from tqdm import tqdm
import pandas as pd

In [2]:
NO_TRIES = 10
NO_N = 15

In [3]:
address = "http://localhost:8413/done"

def single_test(data, NO_TRIES):
    res = []
    for i in tqdm(range(NO_TRIES)):
        r = requests.post(url=address, data = data)
        res.append((r.elapsed, len(r.text)))
    return res

def test(GET_DATA, NO_N, NO_TRIES, filename):
    
    res = {}
    for n in range(NO_N):
        res[n] = single_test(GET_DATA(n), NO_TRIES)

    times = {}
    lens = {}
    for k, v in res.items():
        times[k] = [x[0].seconds + .000001 * x[0].microseconds for x in v]
        lens[k] = [x[1] for x in v]
        
    df_times = pd.DataFrame(times).T
    df_times["mean"] = df_times.mean(1)
    df_times["std"] = df_times.std(1)
    df_times.to_csv(filename + "_times.csv")
                    
    df_lens = pd.DataFrame(lens).T
    df_lens["mean"] = df_lens.mean(1)
    df_lens["std"] = df_lens.std(1)
    df_lens.to_csv(filename + "_lens.csv")
    
    return df_times, df_lens

In [15]:
def linear_data(n):
    return {
        "terminals": "a",
        "nonterminals": "<a>",
        "head": "<a>",
        "rules": "<a> ::= <a> a | a",
        "input": "a " * (2**n),
        "visualization": "none"
    }
    
df = test(linear_data, NO_N, NO_TRIES, "linear")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 334.03it/s]


In [16]:
def ambiguous_data(n):
    return {
        "terminals": "a",
        "nonterminals": "<a>",
        "head": "<a>",
        "rules": "<a> ::= <a> <a> | a",
        "input": "a " * (2**n),
        "visualization": "none"
    }
df = test(ambiguous_data, 4, NO_TRIES, "ambiguous")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 24.22it/s]


In [17]:
print(df[1])

         0        1        2        3        4        5        6        7  \
0     5666     5666     5666     5666     5666     5666     5666     5666   
1     6896     6896     6896     6896     6896     6896     6896     6896   
2    27704    27704    27704    27704    27704    27704    27704    27704   
3  4210600  4210600  4210600  4210600  4210600  4210600  4210600  4210600   

         8        9       mean  std  
0     5666     5666     5666.0  0.0  
1     6896     6896     6896.0  0.0  
2    27704    27704    27704.0  0.0  
3  4210600  4210600  4210600.0  0.0  


In [4]:
def get_data(DEPTH, LENGHT, MODE):
    def get_rule(head, depth,  LENGHT):
        res = head + " ::= "
        for l in range(LENGHT):
            res += "<A" + str(depth+1) + "_" + str(l) + "> | "
        return res[:-3]


    rules = get_rule("<S>",0, LENGHT) + " | x"
    nonterms = "<S> "
    for d in range(DEPTH):
        for l in range(LENGHT):
            head = "<A"+str(d)+"_"+str(l)+">"
            rules += "\n" + get_rule(head, d, LENGHT)
            nonterms += head + " "

    for l in range(LENGHT):
        head = "<A"+str(DEPTH)+"_"+str(l)+">"
        rules += "\n" + head + " ::= y"
        nonterms += head + " "

    return {
        "terminals": "x y",
        "nonterminals": nonterms,
        "head": "<S>",
        "rules": rules,
        "input": "y",
        "visualization": MODE
    }
    

In [5]:
def data_vis_simple(n):
    return get_data(n,n,"simple")

df = test(data_vis_simple, 7, NO_TRIES, "vis_simple")

 10%|████████████████▊                                                                                                                                                       | 1/10 [00:28<04:16, 28.50s/it]


KeyboardInterrupt: 

In [8]:
def data_vis_complex(n):
    return get_data(n,n,"complex")

df = test(data_vis_complex, 7, NO_TRIES, "vis_complex")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:10<00:00,  7.08s/it]
